In [1]:
import numpy as np

In [ ]:
def estimate_test_size(m=95, n=100, pi=0.5, alpha=0.1, num_simulations=10000):
    """
    Estimate the actual size of the chi-squared test for H0: p = q.
    
    Parameters:
    - m, n: Sample sizes
    - alpha: Nominal significance level
    - pi: True probability under H0 (p = q = pi)
    - num_simulations: Number of simulated datasets
    
    Returns:
    - empirical_size: Proportion of times H0 is rejected
    """
    rejections = 0 

    for _ in range(num_simulations):
        # Simulate data
        S_x = np.random.binomial(m, pi)
        S_y = np.random.binomial(n, pi)
        S = S_x + S_y
        F_x = m - S_x
        F_y = n - S_y
        F = F_x + F_y

        # Compute test statistic
        test_statistic = (m + n)*((S_x * F_y) - (S_y * F_x))**2 / (m * n * S * F)

        # Compute critical value for chi-squared distribution
        critical_value = np.percentile(np.random.chisquare(1, num_simulations), 100 * (1 - alpha))

        # Check if we reject H0
        if test_statistic > critical_value:
            rejections += 1

    # Calculate empirical size
    empirical_size = rejections / num_simulations
    return empirical_size


In [24]:
m = 95
n = 100
pi = np.linspace(0.1, 0.9, 9)
alpha = 0.1
num_simulations = 10000

empirical_sizes = np.zeros(len(pi))
for i in range(len(pi)):
    empirical_sizes[i] = estimate_test_size(m, n, pi[i], alpha, num_simulations)
    print(f"True probability (pi): {pi[i]:.2f}, Empirical size: {empirical_sizes[i]:.4f}")


True probability (pi): 0.10, Empirical size: 0.1031
True probability (pi): 0.20, Empirical size: 0.0994
True probability (pi): 0.30, Empirical size: 0.1019
True probability (pi): 0.40, Empirical size: 0.0960
True probability (pi): 0.50, Empirical size: 0.0986
True probability (pi): 0.60, Empirical size: 0.0929
True probability (pi): 0.70, Empirical size: 0.0985
True probability (pi): 0.80, Empirical size: 0.1000
True probability (pi): 0.90, Empirical size: 0.1057


In [ ]:
np.mean(empirical_sizes)

0.09956666666666666